In [1]:
from units.delta_combination import delta_combination
from units.word_generate import generate_sentence
from units.sentence_compare import multi_round_compare
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn

D:\ANACONDA\envs\FYP2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
from units.optimization.GPT import paraphrase

In [18]:
def Tokens_counters(row):
    # Input: Row of dataframe to check the tokens number of each row.
    sentence = row["sentence"]
    total_number = 0
    # Tokenize the sentence to properly handle words and punctuation
    words = word_tokenize(sentence)
    for word in words:
        # Check if the token is an alphanumeric word (to exclude punctuation)
        if word.isalnum():
            if len(wn.synsets(word)) >= 2:
                total_number += 1
    return total_number

In [22]:
# 使用更高效的方式处理 DataFrame 的函数
def expand_dataframe_with_paraphrases(df):
    # 用于收集新行的列表
    new_rows = []
    for _, row in df.iterrows():
        original_sentence = row['sentence']
        original_id = row['ID']
        # 将原始句子作为第一条记录添加
        new_rows.append({'sentence': original_sentence, 'ID': original_id})
        # 获取改写的句子
        paraphrased_sentences = paraphrase(original_sentence)
        # 将改写的句子作为新记录添加
        for paraphrase_sentence in paraphrased_sentences:
            new_rows.append({'sentence': paraphrase_sentence, 'ID': original_id})
    # 创建一个新的 DataFrame
    expanded_df = pd.DataFrame(new_rows)
    return expanded_df

In [20]:
Data = pd.read_excel("Data\\opt\\Rowdata.xlsx")

In [23]:
Data_paraphed = expand_dataframe_with_paraphrases(Data)

,sentence,ID
0,"My daughter uses it for games, email, facebook...",1
1,"My daughter uses it for playing games, composi...",1
2,"Games, email, Facebook, pictures and music are...",1
3,"My daughter uses it for playing games, composi...",1
4,"She uses it for playing games, sending emails,...",1


In [24]:
Data_paraphed["Token Count"] = Data_paraphed.apply(Tokens_counters, axis=1)

In [27]:
Data_paraphed = pd.read_excel("Data\\opt\\Data_opt.xlsx")

In [30]:
def cal_similarity_sentences(row):
    print("输入句子: ",row["sentence"],"\n理解程度: ",row["Delta"],"\n换词数量: ",row["Token Count"])
    sender_words, reveiver_words, receiver_selection, word_similarity = delta_combination(row["sentence"], row["Delta"])
    similarity = multi_round_compare(sender_words, reveiver_words, row["Token Count"])
    print("双边理解相似度: ",similarity)
    print("###################################################################")
    return similarity

In [ ]:
Data_paraphed = pd.read_excel("Data\\opt\\OPT2.xlsx")
Data_paraphed["sim"] = Data_paraphed.apply(cal_similarity_sentences, axis=1)

Data.to_excel("Result\\result_5t_to_20t_OPT2.xlsx")

输入句子:  My only regret is that I cannot control the touchpad using a button, unlike my big 19" laptop. 
理解程度:  0 
换词数量:  12
双边理解相似度:  0.2016263861209154
###################################################################
输入句子:  If only I could switch off the touchpad with a button, just like my big 19" laptop. 
理解程度:  0 
换词数量:  10
双边理解相似度:  0.09234748538583518
###################################################################
输入句子:  The touchpad on my big 19" laptop is the only option I would prefer to have, as it can be turned off with a button. 
理解程度:  0 
换词数量:  16
双边理解相似度:  0.21393003286793827
###################################################################
输入句子:  Unlike my big 19" laptop, I only wish there was a button-less touchpad available on this. 
理解程度:  0 
换词数量:  11
双边理解相似度:  0.31463738307356837
###################################################################
输入句子:  Although it's not practical, I would have preferred to have the touchpad turned off with a button, just l